# BlackBelt Consulting

## 1. Benson Project

### 1.1 Data Collection

In [1]:
import pandas as pd
import datetime

In [2]:
numdays = 353  # setting the number of days we want to take back 1 week from 06-29
# We want data for this year (1st semester) and the last quarter of 2018 (so we can analyze the holiday's season)

In [3]:
date_time_str = '2019-06-22 08:15:27.243860'  # penultimate date of the range in the MTA website
date_time_obj = datetime.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S.%f')  
# transforming it to a date_time object
date_time_obj

datetime.datetime(2019, 6, 22, 8, 15, 27, 243860)

In [4]:
date_list = [date_time_obj - datetime.timedelta(days=x) for x in range(0, numdays, 7)]
# creating a list of dates that starts on the penultimate date and goes back the numdays we've set 
# (jumping 7 days each time)
date_list

[datetime.datetime(2019, 6, 22, 8, 15, 27, 243860),
 datetime.datetime(2019, 6, 15, 8, 15, 27, 243860),
 datetime.datetime(2019, 6, 8, 8, 15, 27, 243860),
 datetime.datetime(2019, 6, 1, 8, 15, 27, 243860),
 datetime.datetime(2019, 5, 25, 8, 15, 27, 243860),
 datetime.datetime(2019, 5, 18, 8, 15, 27, 243860),
 datetime.datetime(2019, 5, 11, 8, 15, 27, 243860),
 datetime.datetime(2019, 5, 4, 8, 15, 27, 243860),
 datetime.datetime(2019, 4, 27, 8, 15, 27, 243860),
 datetime.datetime(2019, 4, 20, 8, 15, 27, 243860),
 datetime.datetime(2019, 4, 13, 8, 15, 27, 243860),
 datetime.datetime(2019, 4, 6, 8, 15, 27, 243860),
 datetime.datetime(2019, 3, 30, 8, 15, 27, 243860),
 datetime.datetime(2019, 3, 23, 8, 15, 27, 243860),
 datetime.datetime(2019, 3, 16, 8, 15, 27, 243860),
 datetime.datetime(2019, 3, 9, 8, 15, 27, 243860),
 datetime.datetime(2019, 3, 2, 8, 15, 27, 243860),
 datetime.datetime(2019, 2, 23, 8, 15, 27, 243860),
 datetime.datetime(2019, 2, 16, 8, 15, 27, 243860),
 datetime.datetime

In [5]:
# transforming the dates into strings and putting in a list:

url_dates = []

for i in date_list:
    year = str(i.year).replace("20", "")
    day = str(i.day)
    if i.day < 10:
        day = "0" + day
    month = str(i.month)
    if i.month < 10:
        month = "0" + month
    date_str = year + month + day
    url_dates.append(date_str)

url_dates[:5]

['190622', '190615', '190608', '190601', '190525']

In [6]:
# starting the dataframe with the last available date, which is june 29, 19:
df = pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_190629.txt")
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2019,00:00:00,REGULAR,7107725,2407457
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2019,04:00:00,REGULAR,7107738,2407465
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2019,08:00:00,REGULAR,7107761,2407491
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2019,12:00:00,REGULAR,7107858,2407541
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2019,16:00:00,REGULAR,7108075,2407581


In [ ]:
# concatenating each new date to the original dataframe:
for url in url_dates:
    df2 = pd.read_csv(
        "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt".format(url))
    df = pd.concat([df, df2], ignore_index=True)

In [ ]:
df.tail()

In [ ]:
df.info(null_counts=True)

### 1.2 Data Treatment

In [ ]:
# First, let's transform the date and time columns into one column as a datetime object:
df["DATE_TIME"] = pd.to_datetime(df.DATE + " " + df.TIME, 
                                            format="%m/%d/%Y %H:%M:%S")

In [ ]:
df.info()

In [ ]:
sorted(df["DATE"].unique())

In [ ]:
df.columns = [column.strip() for column in df.columns]

In [ ]:
df["TURNSTILE_ID"] = df["C/A"] + " " + df["UNIT"] + " " + df["SCP"] + " " + df["STATION"]

In [ ]:
df.head()

In [ ]:
# Checking if there are readings that are duplicates
(df
 .groupby(["TURNSTILE_ID", "DATE_TIME"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False)).head(50)

In [ ]:
# On some days, we seem to have two entries for same time.  Let's take a look at a couple of examples:
mask = ((df["TURNSTILE_ID"] == "N071 R013 00-00-00 34 ST-PENN STA") &
(df["DATE_TIME"].dt.date == datetime.datetime(2019, 2, 27).date()))

df[mask]

On this case, the RECOVR AUD seems to be the correct one since the entries and exits values are similar to the other readings.

In [ ]:
# Other example:
mask = ((df["TURNSTILE_ID"] == "R205A R014 04-02-01 FULTON ST") &
(df["DATE_TIME"].dt.date == datetime.datetime(2018, 10, 21).date()))

df[mask]

On the other hand, in this case, the RECOVR AUD seems to be the wrong one (exits are much larger than the regular exits).

In [ ]:
# Since we can't map every one of these ocurrencies, we will remove all the duplicates and try to deal with the 
# aparent register errors in another way later:
df.sort_values(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], 
                          inplace=True, ascending=False)
df.drop_duplicates(subset=["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], inplace=True)

In [ ]:
# No duplicate problems anymore:
(df
 .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False)).head(15)

In [ ]:
df.describe()

For our data we are aiming only in the entries since we believe this is a better approach to gauge the number of people that may be focusing on the ads. 

We can see from the table above that the average of cumulative exits is much lower than the entries. This probably happens because many people don't use the turnstiles to exit a station.

Also, people entering the stations are more likely to spend more time inside it than people exiting the station (waiting for the subway to arrive) and therefore, more chances to see the digital banners.

In [ ]:
# Conclusion: Drop Exits and Desc Column.  To prevent errors in multiple run of cell, errors on drop is ignored
df = df.drop(["EXITS", "DESC"], axis=1, errors="ignore")

In [ ]:
df.reset_index(drop=True)

In [ ]:
# There seems to be a problem with 23rd st station (and others) because different stations are named the same way:
df[df["STATION"] == "23 ST"].groupby(["LINENAME"]).mean()

Let's try to solve this by changing only the name of the most important problematic stations (the ones that probably have a high foot traffic and may mess up our results) into "station + line".

We'll do this for:
- 23 st
- 86 st
- 96 st
- 14 st
- 125 st
- Chambers St
- 50 st
- Canal St
- 28 St
- 72 St

In [ ]:
# Let's create a new dataframe with only the problematic stations and remove them from the original df.
prob = ((df["STATION"] == '14 ST') |
(df["STATION"] == '23 ST') | 
(df["STATION"] == '86 ST') |
(df["STATION"] == '96 ST') |
(df["STATION"] == 'CANAL ST') |
(df["STATION"] == 'CHAMBERS ST') |
(df["STATION"] == '50 ST') |
(df["STATION"] == '28 ST') |
(df["STATION"] == '72 ST') |
(df["STATION"] == '125 ST'))
df_prob = df[prob]

In [ ]:
df.drop(df[prob].index, inplace=True)
sorted(df["STATION"].unique())

In [ ]:
# For this new df, let's change the STATION to STATION + LINENAME:

df_prob["STATION"] = df_prob["STATION"] + " " + df_prob["LINENAME"]
df_prob.head()

In [ ]:
# Now, let's concatenate this modified df into our original df to bring these stations back:
df = pd.concat([df, df_prob], ignore_index=True)

In [ ]:
sorted(df["STATION"].unique())

Now, let's try to calculate the number of real entries in each period of time, since the original data only shows the cumulative entries registered up to that time:


In [ ]:
df = df.groupby(["STATION","TURNSTILE_ID", "LINENAME", "DATE", "TIME", "DATE_TIME"],as_index=False).ENTRIES.sum()

In [ ]:
df[["PREV_DATETIME", "PREV_ENTRIES"]] = (df
                                                       .groupby(["STATION", "TURNSTILE_ID", "LINENAME"])["DATE_TIME", "ENTRIES"]
                                                       .apply(lambda grp: grp.shift(1)))  

In [ ]:
df.head(10)

In [ ]:
# Drop the rows for the earliest date_time in the df
df.dropna(subset=["PREV_DATETIME"], axis=0, inplace=True)

Another issue we found was the fact that the entries counters usually count up, eventually hit some number, and reset to 0. Also, sometimes they also count down as a reverse counter. To solve this, let's use the following function:


In [ ]:
def get_daily_counts(row, max_counter):
    counter = row["ENTRIES"] - row["PREV_ENTRIES"]
    if counter < 0:
        # Maybe counter is reversed?
        counter = -counter
    if counter > max_counter:
        # Maybe counter was reset to 0? 
        print(row["ENTRIES"], row["PREV_ENTRIES"])
        counter = min(row["ENTRIES"], row["PREV_ENTRIES"])
    if counter > max_counter:
        # Check it again to make sure we're not still giving a counter that's too big
        return 0
    return counter

# If counter is > 10k, then the counter might have been reset.
# 10k seems to be a reasonable number for maximum people entering in a period of hours.
# Just set it to zero as different counters have different cycle limits
df["REAL_ENTRIES"] = df.apply(get_daily_counts, axis=1, max_counter=10000)

In [ ]:
df.head()

In [ ]:
df[df["STATION"] == "GRD CNTRL-42 ST"].sort_values(by=["REAL_ENTRIES"],ascending=False)

In [ ]:
df["WEEKDAY"] = df["DATE_TIME"].dt.weekday_name
df.head()

In [ ]:
df = df.assign(SESSION=pd.cut(df.DATE_TIME.dt.hour,[-1,5,11,17,23],labels=['Dawn','Morning','Afternoon','Evening']))
df

In [ ]:
df[df["SESSION"] == "Evening"]

### 1.3 Exploratory Data Analysis

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()  # automatic seaborn settings

%matplotlib inline

In [ ]:
sns.distplot(df['REAL_ENTRIES']
             [df['REAL_ENTRIES'] < 2000]);

In [ ]:
# Let's first get the daily entries by station:
stations_daily = \
    (df.groupby(['STATION','DATE',"WEEKDAY"])['REAL_ENTRIES'].sum()
                 .reset_index())  

stations_daily.head()

In [ ]:
# To discover the stations with more entries:
stations = \
    (stations_daily.groupby(['STATION'])['REAL_ENTRIES'].sum()
                   .reset_index()
                   .sort_values(by='REAL_ENTRIES',ascending=False))

stations.head(20)

In [ ]:
# To get top 15 stations by daily volume (sum across all days is a reasonable way to define this):
top15_stations = \
    (stations_daily.groupby(['STATION'])['REAL_ENTRIES'].sum()
                   .reset_index()
                   .sort_values(by='REAL_ENTRIES',ascending=False) 
                   .STATION.head(15))

top15_stations

In [ ]:
# next create a new df that filters the stations daily data down to the top 15 stations
stations_daily_top15 = \
    stations_daily[stations_daily['STATION'].isin(top15_stations)]
stations_daily_top15.sort_values(by = 'REAL_ENTRIES')

In [ ]:
# use seaborn to create a boxplot by station
sns.boxplot('REAL_ENTRIES', 'STATION', data=stations_daily_top15)

In [ ]:
avg_per_day = stations_daily_top15.groupby(["STATION"])["REAL_ENTRIES"].mean()
avg_per_day = avg_per_day.to_frame().reset_index()
avg_per_day

In [ ]:
plt.figure(figsize=(15,8))
sns.barplot('STATION', 'REAL_ENTRIES', data=avg_per_day.sort_values(['REAL_ENTRIES'], ascending=False), palette ="Blues_d");

plt.xticks(rotation = 'vertical');
plt.ylabel("AVERAGE DAILY ENTRIES", fontsize=14);
plt.xlabel("STATION", fontsize=14)
plt.title("TOP 15 - HIGH FOOT TRAFFIC STATIONS", fontsize=20);
plt.savefig("plot1.svg", bbox_inches='tight')
plt.savefig("plot1.png", bbox_inches='tight')

In [ ]:
# Now let's try to find more patterns:

entries_by_weekday = df.groupby(['STATION',"WEEKDAY"])['REAL_ENTRIES'].mean()
entries_by_weekday

In [ ]:
entries_by_weekday = entries_by_weekday.unstack(level = -1).reset_index()
entries_by_weekday.head()

In [ ]:
entries_weekday_top15 = \
    entries_by_weekday[entries_by_weekday['STATION'].isin(top15_stations)]
entries_weekday_top15

In [ ]:
avg_weekday_top15 = entries_weekday_top15.groupby('STATION').mean()
avg_weekday_top15

In [ ]:
cols = avg_weekday_top15.columns.tolist()
cols

In [ ]:
cols = cols[1] + " " + cols[5] + " " + cols[6] + " " + cols[4] + " " + cols[0]+ " " + cols[2] + " " + cols[3]

In [ ]:
cols = cols.split()
cols

In [ ]:
avg_weekday_top15 = avg_weekday_top15[cols]

avg_weekday_top15

In [ ]:
plt.figure(figsize = (15,10))
sns.heatmap(avg_weekday_top15,annot=False, cmap="Reds");
plt.xlabel("WEEKDAY", fontsize=14);
plt.ylabel("STATION", fontsize=14);
plt.title("AVERAGE ENTRIES PER WEEKDAY", fontsize=20)
plt.savefig("plot2.svg")
plt.savefig("plot2.png")
# plt.xticks(ticks=[0,1,2,3,4,5,6], labels=[0,1,2,3,4,5,6]);


In [ ]:
top_15_stations = df[df['STATION'].isin(top15_stations)]

In [ ]:
avg_by_session = top_15_stations.groupby(['STATION',"SESSION"])['REAL_ENTRIES'].mean()

In [ ]:
avg_by_session = avg_by_session.unstack(level = -1)
avg_by_session

In [ ]:
plt.figure(figsize = (15,10))
sns.heatmap(avg_by_session,annot=False, cmap="Greens");
plt.xlabel("TIME OF DAY", fontsize=14);
plt.ylabel("STATION", fontsize=14);
plt.title("AVERAGE ENTRIES PER TIME OF DAY", fontsize=20)
plt.savefig("plot3.svg")
plt.savefig("plot3.png")

In [ ]:
daily_entries_15 = top_15_stations.groupby(["STATION", "DATE_TIME"])["REAL_ENTRIES"].sum()
daily_entries_15

In [ ]:
daily_entries_15 = daily_entries_15.to_frame().reset_index()

In [ ]:
daily_entries_15['DATE'] = daily_entries_15['DATE_TIME'].dt.date

In [ ]:
daily_entries_15 = daily_entries_15.groupby(["STATION", "DATE"])["REAL_ENTRIES"].sum()

In [ ]:
daily_entries_15 = daily_entries_15.to_frame().reset_index()
daily_entries_15.head()

In [ ]:
daily_entries_15["DATE"] = pd.to_datetime(daily_entries_15["DATE"]) 
daily_entries_15.info()

In [ ]:
daily_entries_15["MONTH"] = daily_entries_15["DATE"].dt.month
daily_entries_15.head()

In [ ]:
daily_entries_15.info()

In [ ]:
daily_entries_15["SEASON"] = "NS"

# Let's create a new dataframe with only the problematic stations and remove them from the original df.
summer = ((daily_entries_15["MONTH"] == 6) |
(daily_entries_15["MONTH"] == 7) | 
(daily_entries_15["MONTH"] == 8))

daily_entries_15.loc[summer,"SEASON"] = "Summer"

In [ ]:
winter = ((daily_entries_15["MONTH"] == 12) |
(daily_entries_15["MONTH"] == 1) | 
(daily_entries_15["MONTH"] == 2))

daily_entries_15.loc[winter,"SEASON"] = "Winter"

fall = ((daily_entries_15["MONTH"] == 9) |
(daily_entries_15["MONTH"] == 10) | 
(daily_entries_15["MONTH"] == 11))

daily_entries_15.loc[fall,"SEASON"] = "Fall"

spring = ((daily_entries_15["MONTH"] == 3) |
(daily_entries_15["MONTH"] == 4) | 
(daily_entries_15["MONTH"] == 5))

daily_entries_15.loc[spring,"SEASON"] = "Spring"

In [ ]:
daily_entries_15

In [ ]:
avg_entries_day = daily_entries_15.groupby("DATE")["REAL_ENTRIES"].mean()
avg_entries_day = avg_entries_day.to_frame().reset_index()
avg_entries_day.head(3)

In [ ]:
avg_entries_day["COLUMN"] = "1"

In [ ]:
seasonal = avg_entries_day.set_index('DATE').groupby('COLUMN')['REAL_ENTRIES'].rolling(30).mean()
seasonal = seasonal.to_frame().reset_index()
seasonal.drop(["COLUMN"],axis=1,inplace=True)
seasonal.dropna(axis=0, inplace=True)

In [ ]:
seasonal.head()

In [ ]:
plt.figure(figsize = (15,10))
plt.plot(seasonal['DATE'],seasonal['REAL_ENTRIES'])
plt.xlabel("DATE", fontsize=14);
plt.ylabel("AVERAGE ENTRIES", fontsize=14);
plt.title("TOP 15 STATIONS (HIGH FOOT TRAFFIC)\nAVERAGE ENTRIES PER DAY - 30 DAYS ROLLING", fontsize=20)
plt.savefig("plot4.svg")
plt.savefig("plot4.png")


In [ ]:
entries_per_month = daily_entries_15.groupby(["STATION", "MONTH"])["REAL_ENTRIES"].mean()
entries_per_month = entries_per_month.to_frame()
entries_per_month = entries_per_month.unstack(-1)
entries_per_month.head()

In [ ]:
entries_per_month = entries_per_month.transpose()

In [ ]:
entries_per_month = (100. * entries_per_month / entries_per_month.sum())

In [ ]:
entries_per_month = entries_per_month.transpose()
entries_per_month

In [ ]:
plt.figure(figsize = (15,10))
sns.heatmap(entries_per_month,annot=False, cmap="Greens");
plt.xlabel("MONTH", fontsize=14);
plt.ylabel("STATION", fontsize=14);
plt.title("AVERAGE ENTRIES PER MONTH", fontsize=20)
plt.savefig("plot5.svg")
plt.savefig("plot5.png")

In [ ]:
entries_per_season = daily_entries_15.groupby(["STATION", "SEASON"])["REAL_ENTRIES"].mean()
entries_per_season = entries_per_season.to_frame()
entries_per_season = entries_per_season.unstack(-1)
entries_per_season.head()

In [ ]:
entries_per_season = entries_per_season.transpose()

In [ ]:
entries_per_season = (100. * entries_per_season / entries_per_season.sum())

In [ ]:
entries_per_season = entries_per_season.transpose()


In [ ]:
# entries_per_season[()'REAL_ENTRIES', 'Winter'), ('REAL_ENTRIES', 'Spring'), ('REAL_ENTRIES', 'Summer'), ('REAL_ENTRIES', 'Fall')]
entries_per_season = entries_per_season['REAL_ENTRIES'][["Winter", 'Spring', 'Summer', 'Fall']]

In [ ]:
plt.figure(figsize = (15,10))
sns.heatmap(entries_per_season,annot=False, cmap="YlOrRd");
plt.xlabel("SEASON", fontsize=14);
plt.ylabel("STATION", fontsize=14);
plt.title("AVERAGE ENTRIES BY SEASON (AS % OF THE WHOLE YEAR)", fontsize=20)
plt.savefig("plot6.svg")
plt.savefig("plot6.png")
# plt.xticks(ticks=[0,1,2,3,4,5,6], labels=[0,1,2,3,4,5,6]);
